<a href="https://colab.research.google.com/github/yongyogn/practice/blob/prac/220712.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 베이즈 정리 실습
import pandas as pd
smoke_data=pd.DataFrame({'나이그룹':['10','10','10','10','10','10','10','10','10','10','20','20','20','20','20','20','20','20','20','20','30','30','30','30','30','30','30','30','30','30',],'흡연여부':['유','유','무','무','무','무','무','무','무','무','유','유','유','유','유','무','무','무','무','무','유','유','유','유','유','유','유','무','무','무',]})
data_crosstab=pd.crosstab(smoke_data['나이그룹'],smoke_data['흡연여부'],margins=False)
print(data_crosstab)

p_10=data_crosstab.loc['10'].sum()/data_crosstab.values.sum() # 전체 표본 중 10대의 비율
print('10대 확률:',p_10)
p_20=data_crosstab.loc['20'].sum()/data_crosstab.values.sum() # 전체 표본 중 20대의 비율
print('20대 확률:',p_20)
p_30=data_crosstab.loc['30'].sum()/data_crosstab.values.sum() # 전체 표본 중 30대의 비율
print('30대 확률:',p_30)
p_smok=data_crosstab['유'].sum()/data_crosstab.values.sum() # 전체 표본 중 흡연자의 비율
print('전체 흡연 확률:',p_smok)
p_smok_non=data_crosstab['무'].sum()/data_crosstab.values.sum() # 전체 표본 중 비흡연자의 비율
print('전체 비흡연 확률:',p_smok_non)

p_smok_given_10=data_crosstab.loc['10']['유']/data_crosstab.loc['10'].sum() # 10대임이 주어졌을 때 흡연자일 확률
print('P(흡연|10대):',p_smok_given_10)
p_smok_given_20=data_crosstab.loc['20']['유']/data_crosstab.loc['20'].sum() # 20대임이 주어졌을 때 흡연자일 확률
print('P(흡연|20대):',p_smok_given_20)
p_smok_given_30=data_crosstab.loc['30']['유']/data_crosstab.loc['30'].sum() # 30대임이 주어졌을 때 흡연자일 확률
print('P(흡연|30대):',p_smok_given_30)

total_p=(p_10*p_smok_given_10)+(p_20*p_smok_given_20)+(p_30*p_smok_given_30)
print('전확률:',total_p)
p_30_smok_given_30=(p_30*p_smok_given_30) # 30이고 흡연할 확률(30대일 확률과 흡연할 확률의 교집합)
print('P(30대)*P(흡연|30대):',p_30_smok_given_30)
p_30_given_smok=p_30_smok_given_30/total_p # 흡연자임이 주어졌을 때 나이가 30대일 사후확률
print('담배피는 사람일 때 30대일 확률:{}'.format(round(p_30_given_smok,3)))

# 나이브 베이즈
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
X,y=load_iris(return_X_y=True)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.5,random_state=0)
nb=GaussianNB()
y_pred=nb.fit(X_train,y_train).predict(X_test)
print(y_pred)
print("Number of mislabeled points out od a total %d points:%d"%(X_test.shape[0],(y_test!=y_pred).sum()))

In [ ]:
# 이산확률분포 실습
import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
if os.name=='nt':
    font_family='Malgun Gothic'
else:
    font_family='AppleGothic'
sns.set(font=font_family,rc={'axes.unicode_minus':False})

# 베르누이 분포
mu=0.7 # 성공확률 0.7
bern=stats.bernoulli(mu)
b_x=[0,1] # 0은 실패, 1은 성공
bern_pmf=bern.pmf(b_x) # 확률질량함수는 성공과 실패 각각의 확률에 대한 그래프
plt.bar(b_x,bern_pmf)
plt.xlim(-1,2)
plt.ylim(0,1)
plt.xticks([0,1],['x=0','x=1'])
plt.xlabel('표본값')
plt.ylabel('P(x)')
plt.title('베르누이 확률변수의 확률질량함수')
plt.show()

x_sample_bern=bern.rvs(100,random_state=0)
x_sample_bern
y_bern=np.bincount(x_sample_bern,minlength=2)/float(len(x_sample_bern)) # 성공 확률이 0.7인 베르누이 분포를 따르는 랜덤 표본 100개 추출
df_bern=pd.DataFrame({'결과':bern_pmf,'검증(시뮬레이션)':y_bern}) # 이론적인 0.3:0.7의 비율과 어느정도 일치하는지 비교
df_bern.index=[0,1]
df_bern

# 이항분포
print('k=3,n=15,p=0.3의 PMF:',stats.binom.pmf(3,n=15,p=0.3)) # 성공확률 0.3인 실험 15회 반복해서 3회 성공할 확률

print('k=4,n=15,p=0.3의 PMF:',stats.binom.cdf(4,n=15,p=0.3)) # 성공확률 0.3인 실험 15회 반복해서 0회~4회 성공할 누적 확률

def bion_dist(k,n,p):
    pmf_bion=stats.binom.pmf(k,n=n,p=p) # 성공확률 p인 실험 n번 반복할 때 k번 성공할 확률
    return pmf_bion
x=np.arange(0,16)
pdf_result=[bion_dist(k,15,0.3) for k in range(0,16)] # 성공횟수가 0부터 15까지일 때 각각의 확률

plt.title('확률질량함수')
plt.ylim(0,0.3)
plt.text(12.5,0.28,'n=15, p=0.3')
plt.bar(x,pdf_result,color='lightcoral')

# 포아송분포 실습
stats.poisson.pmf(3,2,loc=0) # 평균 발생 횟수가 2인 사건이 3회 일어날 확률

def pois_dist(n,lamb):
    pmf_poi=stats.poisson.pmf(n,lamb,loc=0) # 평균 발생 횟수가 lambda인 사건이 n회 일어날 확률
    return pmf_poi
x=np.arange(0,41)
poi_pdf_result=[pois_dist(n,10) for n in range(0,41)] # 성공횟수가 0부터 40까지일 때 각각의 확률
plt.ylim(0,0.15)
plt.text(33.5,0.14,'람다=10')
plt.bar(x,poi_pdf_result)
plt.show()

x=np.arange(0,41)
fig=plt.figure(figsize=(10,10))
fig.add_subplot(3,1,1)
pmf_poin1=np.array([pois_dist(n,10) for n in range(0,41)])
plt.ylim(0,0.15)
plt.text(33.5,0.12,'람다=10')
plt.bar(x,pmf_poin1,color='lightcoral')

fig.add_subplot(3,1,2)
pmf_poin1=np.array([pois_dist(n,15) for n in range(0,41)])
plt.ylim(0,0.15)
plt.text(33.5,0.12,'람다=15')
plt.bar(x,pmf_poin1,color='mediumaquamarine')

fig.add_subplot(3,1,3)
pmf_poin1=np.array([pois_dist(n,20) for n in range(0,41)])
plt.ylim(0,0.15)
plt.text(33.5,0.12,'람다=20')
plt.bar(x,pmf_poin1,color='royalblue')

In [ ]:
# 연속확률분포 실습
import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
if os.name=='nt':
    font_family='Malgun Gothic'
else:
    font_family='AppleGothic'
sns.set(font=font_family,rc={'axes.unicode_minus':False})

# 정규분포
norm=stats.norm(loc=70,scale=10)
s_x=np.linspace(-10,150,2000)
s_pdf=norm.pdf(s_x)

plt.plot(s_x,s_pdf)
plt.title('정규분포의 확률밀도함수')
plt.xlabel('$x$')
plt.ylabel('$p(x)$')
plt.show()

s_cdf=norm.cdf(s_x)
plt.plot(s_x,s_cdf)
plt.title('누적분포함수')
plt.xlabel('$x$')
plt.ylabel('$F(x)$')
plt.show()

# 표준정규분포
stan_norm=stats.norm(loc=0,scale=1)
sn_x=np.linspace(-5,5,1000)
sn_pdf=stan_norm.pdf(sn_x)

plt.plot(sn_x,sn_pdf)
plt.title('표준정규분포의 확률밀도함수')
plt.xlabel('$x$')
plt.ylabel('$p(x)$')
plt.show()

# 표준화
n_data=np.random.randint(1,100,size=1000)
n_data=np.sort(n_data)
n_data.shape
n_mean=np.mean(n_data)
print('mean: ',np.mean(n_data))
n_std=np.std(n_data)
print('standard deviation:',np.std(n_data))

norm_n_data=stats.norm(loc=n_mean,scale=n_std)
n_pdf=norm_n_data.pdf(n_data)
n_data.shape

plt.plot(n_data,n_pdf)
plt.title('정규분포의 확률밀도함수')
plt.xlabel('$x$')
plt.ylabel('$p(x)$')
plt.show()

z_data=list(n_data)
z_data_list=[]
for i in z_data:
    z=(i-n_mean)/n_std
    z_data_list.append(z)
norm_z_data=stats.norm(loc=0,scale=1)
z_pdf=norm_z_data.pdf(z_data_list)

plt.plot(z_data_list,z_pdf)
plt.title('표준정규분포의 확률밀도함수')
plt.xlabel('$x$')
plt.ylabel('$p(x)$')
plt.show()

# 카이제곱분포
from scipy.stats import chi2
Chi_X=np.linspace(0.5,50,100)
plt.figure(figsize=(10,5))
for dof in [1,2,3,4,5,10,20,30,40,50]:
    plt.plot(Chi_X,stats.chi2(dof).pdf(Chi_X),label=r'$\chi^2$('+str(dof)+')')
plt.xlabel(r'$\chi^2$')
plt.ylabel('y')
plt.grid()
plt.title(r'$\chi^2$ Distribution with scipy.stats')
plt.legend()
plt.show()

x_o=[324,78,261]
x_e=[371,80,212]
x_crosstab=pd.DataFrame([x_o,x_e],columns=['서울','인천','대구'],index=['관찰도수','기대도수'])
x_crosstab
from scipy.stats import chisquare
result=chisquare(x_o,f_exp=x_e)
result

from scipy.stats import chi2
dof=2
x_chi_l=np.linspace(0,20,201)
pdf_chi=chi2(dof).pdf(x_chi_l)
x95_c=chi2(dof).ppf(.95)
plt.plot(x_chi_l,pdf_chi,'b--')
plt.axvline(x=x95_c,color='lightcoral',linestyle=':')
plt.text(x95_c,.4,'임계값='+str(round(x95_c,4)),horizontalalignment='center',color='red')
plt.axvline(x=result[0],color='purple',linestyle=':')
plt.text(result[0],.4,'p_value'+str(round(result[0],4)),horizontalalignment='center',color='purple')
plt.grid()
plt.show()

In [ ]:
# 표본분포 실습
import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
if os.name=='nt':
    font_family='Malgun Gothic'
else:
    font_family='AppleGothic'
sns.set(font=font_family,rc={'axes.unicode_minus':False})

# 표본평균의 표본분포
s_x=np.linspace(-5,100,1000)
s_df=pd.DataFrame(s_x)
s_df_sqz=s_df.squeeze('columns')
sample_data=pd.DataFrame({'dist':s_df_sqz.sample(1000),'type':'Data',})
sample_mean_05=pd.DataFrame({'dist':[s_df_sqz.sample(5).mean() for _ in range(1000)],'type':'5개 샘플평균',})
sample_mean_10=pd.DataFrame({'dist':[s_df_sqz.sample(10).mean() for _ in range(1000)],'type':'10개 샘플평균',})
sample_mean_20=pd.DataFrame({'dist':[s_df_sqz.sample(20).mean() for _ in range(1000)],'type':'20개 샘플평균',})
sample_mean_30=pd.DataFrame({'dist':[s_df_sqz.sample(30).mean() for _ in range(1000)],'type':'30개 샘플평균',})
results=pd.concat([sample_data,sample_mean_05,sample_mean_10,sample_mean_20,sample_mean_30])
g=sns.FacetGrid(results,col='type',col_wrap=1,height=2,aspect=2)
g.map(plt.hist,'dist',range=[-5,100],bins=10)
g.set_axis_labels('dist','Count')
g.set_titles('{col_name}')
plt.tight_layout()
plt.show()

# 중심극한정리
norm=stats.norm(loc=50,scale=10)
s_x=np.linspace(-10,120,2000)
s_pdf=norm.pdf(s_x)
plt.plot(s_x,s_pdf)
plt.title('정규분포의 확률밀도함수')
plt.xlabel('$x$')
plt.ylabel('$p(x)$')
plt.show()

bag_of_mean=[]
for i in range(10):
    m=np.random.binomial(n=1,p=0.5,size=100).mean()
    bag_of_mean.append(m)
pd.DataFrame(bag_of_mean).plot.hist()

bag_of_mean=[]
for i in range(500):
    m=np.random.binomial(n=1,p=0.5,size=100).mean()
    bag_of_mean.append(m)
pd.DataFrame(bag_of_mean).plot.hist()

bag_of_mean=[]
for i in range(1000):
    m=np.random.binomial(n=1,p=0.5,size=100).mean()
    bag_of_mean.append(m)
pd.DataFrame(bag_of_mean).plot.hist()

# QQ플롯 정규성 검정
fig,ax=plt.subplots(figsize=(4,4))
stats.probplot(bag_of_mean,plot=ax)
plt.tight_layout()
plt.show()

# t분포
t_data=np.linspace(-4,4,100)
plt.figure(figsize=(10,6))
for dof in [1,2,5,10,20]:
    t_dist=stats.t(df=dof)
    plt.plot(t_data,t_dist.pdf(t_data),label=('자유도(dof=%d)'%dof))
plt.plot(t_data,stats.norm().pdf(t_data),label='Normal',lw=5,alpha=0.5)
plt.title('자유도에 따른 t분포의 변화')
plt.xlabel('표본값')
plt.ylabel('p(x)')
plt.legend()
plt.show()

# 카이제곱분포
from scipy.stats import chi2
Chi_X=np.linspace(0.5,50,100)
plt.figure(figsize=(10,6))
for dof in [1,2,3,4,5,10,20,30,40,50]:
    plt.plot(Chi_X,stats.chi2(dof).pdf(Chi_X),label=r'$\chi^2$('+str(dof)+')')
plt.xlabel(r'$\chi^2$')
plt.ylabel('y')
plt.grid()
plt.title(r'$\chi^2$ Distribution with scipy.stats')
plt.legend()
plt.show()

# F분포
f_data=np.linspace(0.05,3,1000)
plt.figure(figsize=(10,6))
plt.plot(f_data,stats.f(1,2).pdf(f_data),ls='-',label='F(1,2)')
plt.plot(f_data,stats.f(3,5).pdf(f_data),ls='--',label='F(3,5)')
plt.plot(f_data,stats.f(10,10).pdf(f_data),ls='-.',label='F(10,10)')
plt.axvline(1,ls=':')
plt.xlabel('x')
plt.title('자유도에 따른 F분포의 모양')
plt.legend()
plt.show()

In [ ]:
# 점추정 실습
import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import math
%matplotlib inline
import os
if os.name=='nt':
    font_family='Malgun Gothic'
else:
    font_family='AppleGothic'
sns.set(font=font_family,rc={'axes.unicode_minus':False})

np.random.seed(100)
math_score=pd.DataFrame(np.random.randint(25,101,size=500),columns=['점수'])
math_score.describe()
scores=np.array(math_score['점수'])
scores[:5]
sample=np.random.choice(scores,20)
print(sample)
print(sample.mean())
print(sample.var())
print(sample.std())
for i in range(10):
    sample1=np.random.choice(scores,20)
    print(i+1,'표본평균',sample1.mean())
plt.xlabel('score')
plt.ylabel('relative frequency')
plt.hist(scores,bins=100,range=(0,100),density=True)
plt.xlim(20,100)
plt.show()

sample_score=np.random.choice(scores,10000)
plt.hist(sample_score,bins=100,range=(0,100),density=True)
plt.xlim(20,100)
plt.show()

n=20
nSample=100
sample_mean=np.array([np.random.choice(scores,n,).mean() for _ in range(nSample)])
sample_mean

print('모평균:',scores.mean())
print('표본평균의 평균:',sample_mean.mean())
print('모분산:',scores.var())
print('표본평균의 분산:',sample_mean.var())
print('표본평균의 표준편차:',sample_mean.std())
print('표준오차:',scores.std()/math.sqrt(n))